<div>
<img src="Sales.png" width="600" align="left"/>
</div>

# Next Years Sales Goals

## VP of Sales:

In [35]:
import numpy as np
import pandas as pd

from env import *
from acquire import *
from prepare import *

In [36]:
def get_superstore_data():
    filename = 'superstore_data.csv'
    
    if not os.path.exists(filename):
        sql_db = "superstore_db"
        url = get_db_url(sql_db)
        
        query = '''
                    SELECT * 
                    FROM orders
                    JOIN categories USING(`Category ID`)
                    JOIN products USING(`Product ID`)
                    JOIN regions USING(`Region ID`)
                '''

        df = pd.read_sql(query,url)
        df.to_csv(filename, index=False)
        print(f"Saving {filename}...")
        return df
    
    else:
        print(f"File {filename} already exists. Loading {filename}...")
        
        return pd.read_csv(filename,index_col=False)

In [37]:
def clean_superstore_data(df):
    # set datetime data type
    df['Order Date']=pd.to_datetime(df['Order Date'])
    
    # set 'Date' column as index and sorted
    df = df.set_index("Order Date").sort_index()
    
    # feature engineering: added 'month' column using index
    df['month'] = df.index.month
    
    # feature engineering: added 'year' column using index
    df['year'] = df.index.year
    
    # feature engineering: added 'year' column using index
    df['day'] = df.index.day
    
    # rename columns to python naming conventions ie: snake_case
    df.columns = (df.columns
                .str.replace(' ', '_', regex=True)
                .str.lower()
             )
    return df

In [38]:
superstore_df = get_superstore_data()

Saving superstore_data.csv...


In [32]:
superstore_df = clean_superstore_data(superstore_df)

### What should our sales goals for 2018 be?

### Are there any additional metrics should we track?